# 神经网络基础
## 块和层


In [4]:
import torch
from torch import nn
from torch import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
# Sequential 定义了一个特殊的module
X = torch.rand(2,20) # 生成了一个批量为2 每个样本大小为20的张量
net(X) # 计算前向传播 X为初始数据


tensor([[-1.7333e-02,  1.1162e-01, -3.3055e-02, -5.6004e-02,  1.2141e-04,
          2.2058e-02, -1.6622e-01,  8.5172e-02, -8.4412e-03, -1.8288e-01],
        [-3.5315e-03,  2.6681e-01,  2.0733e-02, -7.7695e-02, -9.3875e-02,
          6.2680e-02, -4.3569e-02,  1.7328e-01, -9.3750e-02, -6.6421e-02]],
       grad_fn=<AddmmBackward0>)

自定义块 实现上文的module

In [7]:
class MLP(nn.Module):  # 继承
    def __init__(self):
        super().__init__() # 调用父类nn.Module的初始化方法
        self.hidden = nn.Linear(20,256)
        self.out  = nn.Linear(256,10)
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))
    

In [8]:
net = MLP() # 实例化
net(X) # pytorch中会有相关的__call__函数来调用，并由对应的优化


AttributeError: module 'torch.functional' has no attribute 'relu'

顺序块实现 Seqential

In [ ]:
class MySequential(nn.Module):
    def __init__(self, *args): # 可变长度的参数列表 表示传入多个模块
        super().__init__()
        for block in args: # 遍历所有模块 并加入self._modules
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0395, -0.0942,  0.0381, -0.2104,  0.3603,  0.1697, -0.0344,  0.0299,
          0.3537,  0.0946],
        [ 0.1011,  0.0478,  0.1215, -0.1353,  0.4766,  0.1194, -0.0874, -0.0552,
          0.3697,  0.0892]], grad_fn=<AddmmBackward0>)

混合搭配

In [ ]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20)) # nn.Seqential 可以是任何的nn.module作为参数
chimera(X)

tensor([[-0.0031,  0.2443, -0.0593,  0.1203, -0.1386, -0.0757, -0.1057, -0.0173,
         -0.1582,  0.1669, -0.2439, -0.1020,  0.0060,  0.0935, -0.3227,  0.1753,
         -0.0891,  0.1451,  0.0798,  0.1560],
        [-0.0126,  0.2569, -0.0237,  0.1198, -0.1504, -0.0811, -0.0775, -0.0198,
         -0.1759,  0.1941, -0.2444, -0.0778,  0.0010,  0.1153, -0.3462,  0.1605,
         -0.0795,  0.1210,  0.0751,  0.1917]], grad_fn=<AddmmBackward0>)

## 参数管理


In [ ]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.0433],
        [0.0941]], grad_fn=<AddmmBackward0>)

参数访问

In [ ]:
print(net[2].state_dict()) # 获取模型中第三个子模块中的所有参数
# 状态机视角 即为该状态下所有的参数

OrderedDict([('weight', tensor([[ 0.2462, -0.1386,  0.2058,  0.1320, -0.3064,  0.0281, -0.2748,  0.1051]])), ('bias', tensor([-0.1039]))])


In [ ]:
print(type(net[2].bias)) # 是一个parameter型的参数，即可学习的
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1039], requires_grad=True)
tensor([-0.1039])


一次性访问所有参数


In [ ]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])
# 使用name


('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


从嵌套块收集参数


In [ ]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4),
                         nn.ReLU())

def block2(): 
    net = nn.Sequential()
    for i in range(4): # 嵌套四个block1
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.0977],
        [0.0977]], grad_fn=<AddmmBackward0>)

In [ ]:
print(rgnet) # 打印网络的形状

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


内置初始化


In [ ]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal) # apply递归的调用net中所有的模型并对其操作
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0025,  0.0005,  0.0038, -0.0069]), tensor(0.))

In [ ]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight) # xavier初始化

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier) # 可以对不同的层使用apply
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.3831, -0.1596,  0.3092, -0.5160])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


参数绑定

In [ ]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared,
                    nn.ReLU(), nn.Linear(8, 1))
net(X) # 不论怎么更新，2和3层的参数都相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## 自定义层

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean() # 均值为0

layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5])) # ()代替forward函数


tensor([-2., -1.,  0.,  1.,  2.])

In [ ]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
# 亦可作为模型块放在更复杂的模型中
Y = net(torch.rand(4, 8))
Y.mean() # 因为浮点数的误差，不会为零

tensor(9.3132e-09, grad_fn=<MeanBackward0>)

自定义类nn.Linear的函数

In [ ]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        # 定义使用Parameter类初始化 所有的可更新的参数都为parameter类
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.3344,  0.2750, -1.7702],
        [-0.4135,  0.4191, -1.4338],
        [-0.2736, -0.0079, -1.0191],
        [ 0.5317, -1.7234,  0.2560],
        [ 0.0073,  0.8304, -0.4371]], requires_grad=True)

正向传播

In [ ]:
linear(torch.rand(2, 5))

tensor([[0.1517, 1.2709, 0.0000],
        [0.0000, 1.9471, 0.0000]])

## 读写文件

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file') # 存在x-file

x2 = torch.load('x-file') # 加载出x-file
x2 

C:\Users\jeff\AppData\Local\Temp\ipykernel_3316\230760058.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x2 = torch.load('x-file') # 加载出x-file


tensor([0, 1, 2, 3])

In [ ]:
y = torch.zeros(4)
torch.save([x, y], 'x-files') # list也可以被存储
x2, y2 = torch.load('x-files') 
(x2, y2)

C:\Users\jeff\AppData\Local\Temp\ipykernel_3316\416286464.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x2, y2 = torch.load('x-files')


(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [ ]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict') # 字典也可以被存储
mydict2 = torch.load('mydict')
mydict2

C:\Users\jeff\AppData\Local\Temp\ipykernel_3316\2269971588.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mydict2 = torch.load('mydict')


{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

存储模型

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

存储

In [ ]:
torch.save(net.state_dict(), 'mlp.params')

加载

In [ ]:
clone = MLP() # 需要携带定义加载
clone.load_state_dict(torch.load('mlp.params'))
clone.eval() # 评估模式
# 对应的是训练模式 会进行dropout操作 并更新batchnorm层

C:\Users\jeff\AppData\Local\Temp\ipykernel_3316\2014828273.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  clone.load_state_dict(torch.load('mlp.params'))


MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [ ]:
Y_clone = clone(X)
Y_clone == Y # 证明被加载了回来

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])